In [2]:
import plotly.express as px
import dash
from dash import dcc, html
import pandas as pd
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from datetime import datetime  
from sqlalchemy import create_engine
import plotly.graph_objs as go
 
# Database connection details
dbname = "amazon_data_weekly"
user = "postgres"
password = "password"
host = "localhost"
 
db_url = f"postgresql+psycopg2://{user}:{password}@{host}/{dbname}"
 
engine = create_engine(db_url)
 
# SQL query to retrieve data
query = "SELECT * FROM merged_data;"
 
# Use pandas to execute the query and retrieve the data
merged_data_df = pd.read_sql_query(query, engine)
 
# Convert datetime columns to be timezone-unaware
for col in merged_data_df.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns:
    merged_data_df.loc[:, col] = merged_data_df[col].dt.tz_localize(None)
 
# Function to map lifecycle state
def map_lifecycle_state(data):
    status_mapping = {
        'End of Life': 'End of Life',
        'Active': 'Active',
        'Ordered': 'Active',
        'Unavailable': 'Active',
        'Unknown State': 'Unknown State'
    }
    data['Lifecycle state'] = data['Lifecycle state'].map(status_mapping).fillna('Unknown State')
    return data
 
def plot_lp_status_weekly(lp_status_weekly, weeks):
    filtered_data = lp_status_weekly[lp_status_weekly['WEEK'].isin(weeks)]
    filtered_data_melted = pd.melt(filtered_data, id_vars=['WEEK'], value_vars=['Active', 'End of Life', 'Unknown State'], var_name='Lifecycle state', value_name='Number of LPs')
 
    fig = px.bar(filtered_data_melted, x='WEEK', y='Number of LPs', color='Lifecycle state', barmode='group',
                 title='Weekly LP Lifecycle Status',
                 labels={'Number of LPs': 'Number of LPs', 'WEEK': 'Week', 'Lifecycle state': 'Lifecycle State'},
                 color_discrete_map={'Active': '#2ca02c', 'End of Life': '#1f77b4', 'Unknown State': '#9467bd'})
   
    fig.update_traces(hovertemplate='%{y}<extra></extra>', base=None)
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=weeks, ticktext=[str(week) for week in weeks], range=[min(weeks), None]),
                      paper_bgcolor='rgba(255, 255, 255, 0.8)')
   
    return fig
 
def plot_toll_transactions(toll_transactions_weekly, weeks):
    filtered_data = toll_transactions_weekly[(toll_transactions_weekly['WEEK'].isin(weeks)) & (toll_transactions_weekly['TRANSACTION TYPE'].isin(['Transponder Toll', 'Plate Toll']))]
    fig = px.bar(filtered_data, x='WEEK', y='Count', color='TRANSACTION TYPE', barmode='group',
                 title=f'Weekly Count of Toll Transactions for Active Assets (Power Units Only)',
                 labels={'Count': 'Number of Transactions', 'WEEK': 'Week', 'TRANSACTION TYPE': 'Transaction Type'},
                 color_discrete_map={'Transponder Toll': '#2ca02c', 'Plate Toll': '#1f77b4'})
 
    fig.update_traces(hovertemplate='%{y}<extra></extra>', base=None)
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=weeks, ticktext=[str(week) for week in weeks]),
                paper_bgcolor='rgba(255, 255, 255, 0.8)'  
    )
 
    return fig
 
def plot_active_lp_sources(active_lp_weekly, weeks):
    filtered_data = active_lp_weekly[(active_lp_weekly['WEEK'].isin(weeks)) & (active_lp_weekly['SOURCE'] != 'CITATION')]
    colors = ['#2ca02c', '#1f77b4', '#9467bd']
    fig = px.bar(filtered_data, x='WEEK', y='Count', color='SOURCE', barmode='group',
                 title=f'Active Status for SRTs VS Electronic Tolls',
                 labels={'Count': 'Number of Active LPs', 'WEEK': 'Week', 'SOURCE': 'Source'},
                 color_discrete_map={source: color for source, color in zip(filtered_data['SOURCE'].unique(), colors)})
 
    fig.update_traces(hovertemplate='%{y}<extra></extra>', base=None)
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=weeks, ticktext=[str(week) for week in weeks]),
                    paper_bgcolor='rgba(255, 255, 255, 0.8)'
   
    )
 
    return fig
 
def plot_sla_trend_bar(sla_trend_df, weeks):
    filtered_data = sla_trend_df[sla_trend_df['WEEK'].isin(weeks)]
    fig = px.bar(filtered_data, x='WEEK', y='Count', color='SLA MET', barmode='group',
                 title=f'SLA Data Week over Week',
                 labels={'Count': 'Number of Transactions', 'WEEK': 'Week', 'SLA MET': 'SLA Status'},
                 color_discrete_map={'Within SLA': '#2ca02c', 'Outside SLA': '#1f77b4'})
 
    fig.update_traces(hovertemplate='%{y}<extra></extra>', base=None)
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=weeks, ticktext=[str(week) for week in weeks]),
                    paper_bgcolor='rgba(255, 255, 255, 0.8)'                
    )
 
    return fig
 
def plot_lp_count_weekly(lp_status_weekly):
    fig = px.line(lp_status_weekly, x='WEEK', y=['Active', 'End of Life','Unknown State'],
                  title='LP Count Week Over Week(Active & End of Life)',
                  labels={'value': 'Number of LPs', 'WEEK': 'Week', 'variable': 'Lifecycle State'},
                  color_discrete_map={'Active': '#2ca02c', 'End of Life': '#1f77b4'})  # Change color mapping here
    fig.update_traces(mode='lines+markers')
    fig.update_layout(paper_bgcolor='rgba(255, 255, 255, 0.8)')
    fig.update_traces(hovertemplate='week %{x} <br>%{y}<extra></extra>')
    return fig
 
def toll_transactions_line_plot(toll_transactions_weekly):
    filtered_data = toll_transactions_weekly[toll_transactions_weekly['TRANSACTION TYPE'].isin(['Transponder Toll', 'Plate Toll'])]
    fig = px.line(filtered_data, x='WEEK', y='Count',
                  title='Count of Active LICENSE PLATEs(Power Units Only) based on Transaction Type',
                  labels={'Count': 'Number of Transactions', 'WEEK': 'Week'},
                  color='TRANSACTION TYPE',
                  color_discrete_map={'Transponder Toll': '#2ca02c', 'Plate Toll': '#1f77b4'})  # Change color mapping here
 
    fig.update_traces(hovertemplate='week %{x} <br>%{y}<extra></extra>')
    fig.update_layout(paper_bgcolor='rgba(255, 255, 255, 0.8)')
 
    return fig
 
 
def plot_active_lp_sources_trend(active_lp_weekly):
    traces = []
    for source, data in active_lp_weekly[active_lp_weekly['SOURCE'] != 'CITATION'].groupby('SOURCE'):
        trace = go.Scatter(x=data['WEEK'], y=data['Count'], mode='lines', name=f'Trend - {source}')
        traces.append(trace)
 
    layout = go.Layout(
        title='Trend of Active LICENSE PLATEs by Source',
        xaxis=dict(title='Week'),
        yaxis=dict(title='Number of Active LPs'),
        paper_bgcolor='rgba(255, 255, 255, 0.8)'
    )
 
    fig = go.Figure(data=traces, layout=layout)
 
    fig.update_traces(hovertemplate='week %{x} <br> %{y}<extra></extra>')
    return fig
 
 
def plot_sla_trend_line(sla_trend_df):
    fig = px.line(sla_trend_df, x='WEEK', y='Count', color='SLA MET',
                  title='SLA Trend Over Time',
                  labels={'Count': 'Number of Transactions', 'WEEK': 'Week', 'SLA MET': 'SLA Status'},
                  color_discrete_map={'Within SLA': '#2ca02c', 'Outside SLA': '#1f77b4'})
    fig.update_traces(mode='lines+markers')
    fig.update_traces(hovertemplate='week %{x} <br>%{y}<extra></extra>')
    fig.update_layout(paper_bgcolor='rgba(255, 255, 255, 0.8)')
 
    return fig
 
 
# Function to calculate savings per week
def calculate_savings(merged_data_df):
    merged_data_df['Savings'] = merged_data_df['HIGH RATES'] - merged_data_df['AMOUNT']
    weekly_savings_summary = merged_data_df.groupby('WEEK')['Savings'].agg(['sum', 'mean']).reset_index()
    weekly_savings_summary.columns = ['Week', 'Total Savings', 'Average Savings']
   
    return weekly_savings_summary
 
# Function to plot savings trend
def plot_savings_trend(weekly_savings_summary):
    fig = px.line(weekly_savings_summary, x='Week', y=['Total Savings', 'Average Savings'], title='Savings Trend Over Time',
                  labels={'value': 'Amount ($)', 'Week': 'Week'},
                  color_discrete_map={'Total Savings': '#1f77b4', 'Average Savings': 'green'})
    fig.update_yaxes(tickprefix="$", tickformat=",")  # Format y-axis tick labels as dollars
    fig.update_traces(mode='lines+markers', hovertemplate='week %{x} <br> $%{y:,.2f}<extra></extra>')  # Update hover information
    fig.update_layout(paper_bgcolor='rgba(255, 255, 255, 0.8)')
    return fig
 
 
def calculate_summary_metrics(merged_data_df, selected_weeks):
    filtered_data = merged_data_df[merged_data_df['WEEK'].isin(selected_weeks)]
 
    total_transactions = filtered_data.shape[0]
    total_savings = filtered_data['Savings'].sum()  # Calculate total savings
    active_lp_count = filtered_data[filtered_data['Lifecycle state'] == 'Active']['LICENSE PLATE'].nunique()
 
    return total_transactions, total_savings, active_lp_count
 
# Function to calculate percentage of transactions within SLA
def calculate_percentage_within_sla(merged_data_df, selected_weeks):
    filtered_data = merged_data_df[(merged_data_df['WEEK'].isin(selected_weeks)) & (merged_data_df['SLA MET'] == 'Within SLA')]
    total_transactions = merged_data_df[merged_data_df['WEEK'].isin(selected_weeks)].shape[0]
    if total_transactions > 0:
        percentage_within_sla = (filtered_data.shape[0] / total_transactions) * 100
    else:
        percentage_within_sla = 0  # Handle case where there are no transactions
    return percentage_within_sla
 
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, 'style.css'])
 
# Define weeks and dropdown options (assuming these are already defined)
unique_weeks = merged_data_df['WEEK'].unique()
unique_weeks.sort()
week_options = [{'label': str(week), 'value': week} for week in unique_weeks]
# Calculate savings
weekly_savings_summary = calculate_savings(merged_data_df)
 
# Group LP status weekly
lp_status_weekly = merged_data_df.groupby(['WEEK', 'Lifecycle state'])['LICENSE PLATE'].nunique().unstack(fill_value=0)
lp_status_weekly = lp_status_weekly.reset_index()
 
# Filter and group toll transactions
# Step 1: Filter for active LICENSE PLATEs and exclude 'TRAILER' report type
active_lp_df = merged_data_df[(merged_data_df['Lifecycle state'] == 'Active') &
                               (merged_data_df['REPORT TYPE'] != 'TRAILER')]
 
# Step 2: Filter for 'Transponder Toll' and 'plate toll' transaction types
toll_transactions_df = active_lp_df[active_lp_df['TRANSACTION TYPE'].isin(['Transponder Toll', 'Plate Toll'])]
# Step 3: Group by week and transaction type, count unique LICENSE PLATEs
toll_transactions_weekly = toll_transactions_df.groupby(['WEEK', 'TRANSACTION TYPE'])['LICENSE PLATE'].nunique().reset_index(name='Count')
 
# Group by week and source
active_lp_df1 = merged_data_df[(merged_data_df['Lifecycle state'] == 'Active')]
active_lp_weekly = active_lp_df1.groupby(['WEEK', 'SOURCE']).size().reset_index(name='Count')
 
# Read data for SLA trend
sla_trend_df = merged_data_df.groupby(['WEEK', 'SLA MET']).size().reset_index(name='Count')
 
unique_weeks = merged_data_df['WEEK'].unique()
unique_weeks.sort()
 
week_options = [{'label': str(week), 'value': week} for week in unique_weeks]
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, 'style.css'])
 
# Modify the layout to remove other filters
app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(
                    html.Img(src="/static/icons/innovative.png", style={'height': '100px'}),
                    style={'text-align': 'center', 'margin-bottom': '20px'}
                ),
            ]
        ),
 
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                dbc.Row(
                                    [
                                        dbc.Col(
                                            dbc.Card(
                                                dbc.CardBody(
                                                    [
                                                        html.Div(
                                                            [
                                                                html.Img(src="/static/icons/TT.png", className='icon'),  # savings icon image
                                                                html.P(id='total-transactions', className='card-text summary-stat')
                                                               
                                                            ],
                                                           
                                                            className='d-flex align-items-center'
                                                        )
                                                    ]
                                                ),
                                                className='card border-light mb-3',
                                            ),
                                            width=2
                                        ),
                                        dbc.Col(
                                            dbc.Card(
                                                dbc.CardBody(
                                                    [
                                                        html.Div(
                                                            [
                                                                html.Img(src="/static/icons/sav.png", className='icon'),  #average savings icon image
                                                                html.P(id='average-savings', className='card-text d-flex align-items-center me-3')
                                                            ],
                                                            className='d-flex align-items-center'
                                                        )
                                                    ]
                                                ),
                                                className='card border-light mb-3'
                                            ),
                                            width=2
                                        ),
                                        dbc.Col(
                                            dbc.Card(
                                                dbc.CardBody(
                                                    [
                                                        html.Div(
                                                            [
                                                                html.Img(src="/static/icons/LP.png", className='icon'),  # license plate icon image
                                                                html.P(id='active-lp-count', className='card-text summary-stat')
                                                            ],
                                                            className='d-flex align-items-center'
                                                        )
                                                    ]
                                                ),
                                                className='card border-light mb-3'
                                            ),
                                            width=2
                                        )
                                    ],
                                    className='summary-row d-flex justify-content-start'
                                )
                            ]
                        ),
                        style={'background-color': '#03082a', 'margin-bottom': '20px'}
                    ),
                    width=12
                ),
            ]
        ),
 
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader(html.H4("Filters", style={'color': '#FFFFFF'})),
                            dbc.CardBody(
                                [
                                    html.Label('Week:', style={'color': '#FFFFFF'}),
                                    dcc.Dropdown(
                                        id='week-dropdown',
                                        options=week_options,
                                        value=unique_weeks[:1],  # Default value to select the first week
                                        multi=True,
                                        style={'margin-bottom': '10px', 'color': 'black'}
                                    ),
                                    html.Label('Lifecycle State:', style={'color': '#FFFFFF'}),
                                    dcc.Dropdown(
                                        id='lifecycle-state-dropdown',
                                        options=[{'label': state, 'value': state} for state in merged_data_df['Lifecycle state'].unique()],
                                        multi=True,
                                        style={'margin-bottom': '10px', 'color': 'black'}
                                    ),
                                    html.Label('Source:', style={'color': '#FFFFFF'}),
                                    dcc.Dropdown(
                                        id='source-dropdown',
                                        options=[{'label': source, 'value': source} for source in merged_data_df['SOURCE'].unique()],
                                        multi=True,
                                        style={'margin-bottom': '10px', 'color': 'black'}
                                    ),
                                    html.Label('Transaction Type:', style={'color': '#FFFFFF'}),
                                    dcc.Dropdown(
                                        id='transaction-type-dropdown',
                                        options=[{'label': transaction, 'value': transaction} for transaction in merged_data_df['TRANSACTION TYPE'].unique()],
                                        multi=True,
                                        style={'margin-bottom': '10px', 'color': 'black'}
                                    ),
                                   
                                ]
                            ),
                        ],
                        style={'background-color': '#03082a', 'margin-bottom': '20px'}
                    ),
 
                    width=2
                ),
                dbc.Col(
                    [
                        dbc.Card(
                            dbc.CardBody(
                                [
                                    html.Div(id='visualization-output', style={'height': '80vh', 'overflow': 'auto'}),
                                    dbc.Button('Show Live Trend', id='plot-toggle-button', n_clicks=0, color='success', className='mt-3'),
                                    dbc.Button('Clear Filters', id='clear-filters-button', n_clicks=0, color='danger', className='mt-3'),
                                ]
                            ),
                            style={'background-color': '#03082a', 'margin-bottom': '20px'}
                        ),
                    ],
                    width=9
                ),
            ],
            style={'background-color': '#03082a'}
        )
    ],
    fluid=True
)
@app.callback(
    [Output('visualization-output', 'children'),
     Output('total-transactions', 'children'),
     Output('average-savings', 'children'),
     Output('active-lp-count', 'children')],
    [Input('plot-toggle-button', 'n_clicks'),
     Input('clear-filters-button', 'n_clicks'),
     Input('week-dropdown', 'value')],
    [State('plot-toggle-button', 'n_clicks')]
)
def update_visualization(button_clicks, clear_clicks, selected_weeks, prev_button_clicks):
    ctx = dash.callback_context
    if not ctx.triggered:
        option_triggered = None
    else:
        option_triggered = ctx.triggered[0]['prop_id'].split('.')[0]
 
    if option_triggered == 'clear-filters-button':
        selected_weeks = unique_weeks[:1]
 
    if not selected_weeks:
        selected_weeks = unique_weeks  # Default to all weeks if none are selected
 
    # Calculate summary metrics
    total_transactions, total_savings, active_lp_count = calculate_summary_metrics(merged_data_df, selected_weeks)
 
    if option_triggered == 'clear-filters-button' or button_clicks % 2 == 0:
        visualization_content = html.Div([
            html.Div([
                dcc.Graph(figure=plot_lp_status_weekly(lp_status_weekly, selected_weeks)),
                dcc.Graph(figure=plot_toll_transactions(toll_transactions_weekly, selected_weeks))
            ],
            style={'display': 'flex', 'flex-direction': 'row', 'justify-content': 'center', 'background-color': '#03082a', 'width': '100%'}),
           
            html.Div([
                dcc.Graph(figure=plot_active_lp_sources(active_lp_weekly, selected_weeks)),
                dcc.Graph(figure=plot_sla_trend_bar(sla_trend_df, selected_weeks))
            ],
            style={'display': 'flex', 'flex-direction': 'row', 'justify-content': 'center', 'background-color': '#03082a', 'width': '100%'}),
         
         ])
    else:
        visualization_content = html.Div([
            html.Div([
                dcc.Graph(figure=plot_lp_count_weekly(lp_status_weekly)),
                dcc.Graph(figure=toll_transactions_line_plot(toll_transactions_weekly))
            ],
            style={'display': 'flex', 'justify-content': 'center', 'background-color': '#03082a', 'width': '100%'}),
 
            html.Div([
                dcc.Graph(figure=plot_active_lp_sources_trend(active_lp_weekly)),
                dcc.Graph(figure=plot_savings_trend(weekly_savings_summary))
            ],
            style={'display': 'flex', 'justify-content': 'center', 'background-color': '#03082a', 'width': '100%'}),
 
            html.Div([
                dcc.Graph(figure=plot_sla_trend_line(sla_trend_df))
            ],
            style={'display': 'flex', 'justify-content': 'center', 'background-color': '#03082a', 'width': '100%'}),
        ])
 
    return visualization_content, f"TOTAL TRANSACTIONS: {total_transactions}", f"TOTAL SAVINGS: ${total_savings:,.2f}", f"ACTIVE LP COUNT: {active_lp_count}"
 
# Update button text based on its state
@app.callback(
    Output('plot-toggle-button', 'children'),
    Input('plot-toggle-button', 'n_clicks')
)  
def update_button_text(n_clicks):
    if n_clicks % 2 == 0:
        return 'Show Line Plot'
    else:
        return 'View Bar Plot'
 
# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=5234)

C:\Users\RamadhanZome\AppData\Local\Temp\ipykernel_11708\2080965507.py:29: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2023-08-10 21:00:00', '2023-08-10 21:00:00', '2023-08-10 21:00:00',
 '2023-08-10 21:00:00', '2023-08-10 21:00:00', '2024-05-31 21:00:00',
 '2024-05-31 21:00:00', '2024-05-31 21:00:00', '2024-05-31 21:00:00',
 '2023-10-18 21:00:00',
 ...
 '2023-08-10 21:00:00', '2023-08-10 21:00:00', '2023-08-10 21:00:00',
 '2023-08-10 21:00:00', '2023-08-10 21:00:00', '2023-08-10 21:00:00',
 '2023-08-10 21:00:00', '2023-08-10 21:00:00', '2023-08-10 21:00:00',
 '2023-08-10 21:00:00']
Length: 1936647, dtype: datetime64[ns]' has dtype incompatible with datetime64[ns, UTC], please explicitly cast to a compatible dtype first.



In [6]:
import folium
from folium.plugins import HeatMap
import pandas as pd

# Sample data: replace with your actual exit location data
data = {
    'latitude': [40.7128, 34.0522, 41.8781, 29.7604, 39.7392],
    'longitude': [-74.0060, -118.2437, -87.6298, -95.3698, -104.9903]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Create a base map centered around the mean of the latitudes and longitudes
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5)

# Prepare the data for the heatmap (list of [latitude, longitude])
heat_data = [[row['latitude'], row['longitude']] for index, row in df.iterrows()]

# Add the heatmap layer
HeatMap(heat_data).add_to(m)

# Save the map as an HTML file
m.save('exit_locations_heatmap.html')

# Display the map
m
